<a href="https://colab.research.google.com/github/hyehyunseo/ESAA_OB/blob/main/0922_%EC%95%99%EC%83%81%EB%B8%94_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**

**[답안]**

직접 투표 분류기는 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 분류기를 말하고, 간접 투표 분류기는 각 클래스의 평균적인 확률 추정값을 계산하여 가장 높은 확률을 가진 클래스를 선택하는 분류기를 말한다.


### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**

**[답안]**

그레디언트 부스팅 앙상블이 훈련 데이터에 과대적합 되었다면, 학습률을 낮게 설정해야한다.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [2]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=0)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다.

In [4]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [6]:
# model fitting
## 랜덤 포레스트 분류기
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
## 엑스트라 트리 분류기
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
extra_trees_clf.fit(X_train, y_train)

ExtraTreesClassifier(random_state=42)

In [8]:
## SVM 분류기
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
svm_clf.fit(X_train, y_train)

LinearSVC(max_iter=100, random_state=42, tol=20)

In [9]:
## MLP 분류기
mlp_clf = MLPClassifier(random_state=42)
mlp_clf.fit(X_train, y_train)

MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [10]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators=[
    ('rf', random_forest_clf),
    ('et', extra_trees_clf),
    ('svm', svm_clf),
    ('mlp', mlp_clf)
], voting='hard')

In [11]:
# 앙상블 모델 훈련
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('et', ExtraTreesClassifier(random_state=42)),
                             ('svm',
                              LinearSVC(max_iter=100, random_state=42, tol=20)),
                             ('mlp', MLPClassifier(random_state=42))])

In [13]:
# 검증 데이터에서 성능 평가
score = voting_clf.score(X_val, y_val)
print(f"앙상블 모델 성능 (검증 데이터): {score:.2f}")

앙상블 모델 성능 (검증 데이터): 0.97


####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [ ]:
# 각 분류 모델 학습
# => 위에서 진행함

In [14]:
# 각 분류 모델의 성능 확인
print('랜덤포레스트에서 검증 데이터 셋의 성능 : ',random_forest_clf.score(X_val, y_val))
print('엑스트라 트리 분류기에서 검증 데이터 셋의 성능 : ',extra_trees_clf.score(X_val, y_val))
print('SVM 분류기에서 검증 데이터 셋의 성능 : ',svm_clf.score(X_val, y_val))
print('MLP 분류기에서 검증 데이터 셋의 성능 : ',mlp_clf.score(X_val, y_val))

랜덤포레스트에서 검증 데이터 셋의 성능 :  0.9676
엑스트라 트리 분류기에서 검증 데이터 셋의 성능 :  0.9702
SVM 분류기에서 검증 데이터 셋의 성능 :  0.8365
MLP 분류기에서 검증 데이터 셋의 성능 :  0.9611


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. SVM 분류기의 성능이 가장 낮다.

In [15]:
# 가장 성능이 낮은 모델 제거
del voting_clf.estimators_[2]

In [16]:
# 제거되었는지 확인하기
voting_clf.estimators_

[RandomForestClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42),
 MLPClassifier(random_state=42)]

In [17]:
# 모델 제거 후 성능 확인
voting_clf.score(X_val, y_val)

0.9717

=> 0.97에서 0.9717로 상승함

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [19]:
# 새 훈련 세트를 저장할 ndarray 생성
X_val_predictions = np.empty((len(X_val), 4), dtype=np.float32)

In [20]:
# 새 훈련 세트 생성
X_val_predictions[:, 0] = random_forest_clf.predict(X_val)
X_val_predictions[:, 1] = extra_trees_clf.predict(X_val)
X_val_predictions[:, 2] = svm_clf.predict(X_val)
X_val_predictions[:, 3] = mlp_clf.predict(X_val)

In [21]:
X_val_predictions

array([[4., 4., 4., 4.],
       [2., 2., 2., 2.],
       [0., 0., 0., 0.],
       ...,
       [7., 7., 7., 7.],
       [1., 1., 1., 1.],
       [6., 6., 2., 6.]], dtype=float32)

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [22]:
rnd_forest = RandomForestClassifier(n_estimators=200, random_state=42)
rnd_forest.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다.

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [23]:
# 새 훈련 세트를 저장할 ndarray 생성
X_test_predictions = np.empty((len(X_val), 4), dtype=np.float32)

In [24]:
# 새 훈련 세트 생성
X_test_predictions[:, 0] = random_forest_clf.predict(X_test)
X_test_predictions[:, 1] = extra_trees_clf.predict(X_test)
X_test_predictions[:, 2] = svm_clf.predict(X_test)
X_test_predictions[:, 3] = mlp_clf.predict(X_test)

In [25]:
X_test_predictions

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [1., 1., 1., 1.],
       ...,
       [1., 1., 1., 1.],
       [3., 3., 3., 3.],
       [0., 0., 0., 0.]], dtype=float32)

In [26]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
y_pred = rnd_forest.predict(X_test_predictions)

In [27]:
# model test
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9694